In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import numpy as np

# El Universal

In [2]:
def universal_scrap_links(url):
    
    links = []
    
    answer = requests.get(url).content        
    
    soup = BeautifulSoup(answer, 'html')        

    titles = soup.select('h3[class="ce3-AutorColumna_Titulo"]', href = True)
    
    for title in titles:
        links.append("https://www.eluniversal.com.mx"+title.select('a[href]')[0]['href'])

    titles = soup.select('[class="ce9-Tipo4_Titulo"]', href = True)
    
    for title in titles:
        links.append("https://www.eluniversal.com.mx"+title.select('a[href]')[0]['href'])
    return links

In [3]:
def universal_scrap_content(link):

    answer = requests.get(link).content

    answer = BeautifulSoup(answer)

    date = answer.select('[class="ce12-DatosArticulo_ElementoFecha"]')[0].text.strip()

    title = answer.select('[class="ceh-Opinion_Titulo"]')[0].text.strip()

    body = answer.select('[class="field field-name-body field-type-text-with-summary field-label-hidden"]')[0].text.strip()

    return title, date, body

In [13]:
def universal_scrap_columnista (url, auth_name):
    titles = []
    dates = []
    bodies = []
    links = []
    
    for link in universal_scrap_links(url):
        title, date, body = universal_scrap_content(link)
        titles.append(title)
        dates.append(date)
        bodies.append(body)
        links.append(link)
    data_dict = {'title':titles, 'date':dates, 'body':bodies, 'link':links}
    df = pd.DataFrame(data_dict)
    df['author'] = auth_name
    df['source'] = 'El Universal'
    return df[['author', 'title', 'date', 'body', 'source','link']]

# El Financiero

In [5]:
def elfinanciero_scrap_links (url):
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(executable_path='../../../Documents/chromedriver', options=options)
    driver.get(url)
    time.sleep(2)
    ActionChains(driver).send_keys(Keys.END).perform()
    time.sleep(2)
    while True:
        try:
            driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div[1]/div[3]/div/button').click()
            time.sleep(2)
            ActionChains(driver).send_keys(Keys.END).perform()
            time.sleep(2)
        except:
            break
    code = driver.page_source
    driver.close()
    soup = BeautifulSoup(code, 'html5lib')
    links = soup.select('div[class="column-title"]')
    links = ['https://www.elfinanciero.com.mx'+link.select('a[href]')[0]['href'] for link in links]
    return links

In [6]:
def elfinanciero_scrap_content(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    
    answer = requests.get(url, headers=headers).content
    
    soup = BeautifulSoup(answer)
    
    paragraphs = soup.select('[class="paragraph"]')
    
    body = ''
    
    for e in paragraphs:
        body += e.text
    
    if len(body)==0:
        body = soup.select('[class="column is-8 content"]')[0].text
        
    if len(body)==0:
        print('aqui murio')
        print(url)
    
    title = soup.select('[class="column is-12-touch is-11-desktop"]')[0].text.replace("\r","")

    date = soup.select('[class="publish"]')[0].text
    
    return title, date, body

In [7]:
def elfinanciero_scrap_columnista (url, auth_name):
    titles = []
    dates = []
    bodies = []
    links = []
    
    for link in elfinanciero_scrap_links(url):
        title, date, body = elfinanciero_scrap_content(link)
        titles.append(title)
        dates.append(date)
        bodies.append(body)
        links.append(link)
    data_dict = {'title':titles, 'date':dates, 'body':bodies, 'link':links}
    df = pd.DataFrame(data_dict)
    df['author'] = auth_name
    df['source'] = 'El Financiero'
    return df[['author', 'title', 'date', 'body', 'source', 'link']]

# Milenio

In [8]:
global driver

def init_driver():
    global driver
    
    options = Options()
    options.add_argument("--headless")
    
    driver = webdriver.Chrome(executable_path='../../../Documents/chromedriver', options=options)
    driver.maximize_window()
    time.sleep(2)
    
    return driver

#driver.get("https://www.milenio.com/opinion/valeria-moy/page/1")

In [9]:
def milenio_url_scrapper(url_1):
    driver = init_driver()
    links = []
    
    driver.get(url_1)
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    try:
        driver.find_element_by_xpath('/html/body/div[4]/div/div/button').click()
    except:
        print('No cookies overlay detected.')
    
    while True:
        print(driver.current_url)
        respuesta = driver.page_source
        sopa = BeautifulSoup(respuesta, 'html')        
        titles = sopa.select('[class="title"]', href = True)
        
        for title in titles:            
            links.append('https://www.milenio.com' + title.select('a[href]')[0]['href'])
        try:
            if driver.current_url.split('/')[-1] == '1':
                driver.find_element_by_xpath('/html/body/div[1]/div/div/div[3]/div/div[2]/div[2]/a').click()
            else:
                driver.find_element_by_xpath('/html/body/div[1]/div/div/div[3]/div/div[2]/div[2]/a[2]').click()
            time.sleep(0.5)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        except Exception as e:
            print(e)
            break
        
    driver.quit()
    return links
    

In [10]:
def milenio_content_scrap(link):
    """"""
    respuesta = requests.get(link).content        
    sopa = BeautifulSoup(respuesta, 'html')      
    title = sopa.select('title')[0].text
    try:
        date = sopa.select('div time[class="date"]')[0].text
        body = sopa.select('div [class="media-container opinion"]', id = 'content-body')[0].text
    except Exception as e:
        #date = sopa.select('body > div.body-content > div > article > div.content-columns > div.column-right > div.nd-content-body > div.content-date > time')[0].text
        return '', '', ''
    
    return title, date, body

In [11]:
def milenio_scrap_columnista (url, auth_name):
    titles = []
    dates = []
    bodies = []
    links2 = []
    contador = 1
    
    links = milenio_url_scrapper(url)
    for link in links:
        title, date, body = milenio_content_scrap(link)
        print(f'Procesando: {contador} de {len(links)}')
        titles.append(title)
        dates.append(date)
        bodies.append(body)
        links2.append(link)
        contador += 1
    data_dict = {'title':titles, 'date':dates, 'body':bodies, 'link':links2}
    df = pd.DataFrame(data_dict)
    df['author'] = auth_name
    df['source'] = 'Milenio'
    return df

In [14]:
#elf_val = elfinanciero_scrap_columnista("https://www.elfinanciero.com.mx/opinion/valeria-moy", 'Valeria Moy')
uni_val = universal_scrap_columnista('https://www.eluniversal.com.mx/valeria-moy/peras-y-manzanas', 'Valeria Moy')
mil_val = milenio_scrap_columnista('https://www.milenio.com/opinion/valeria-moy/page/1', 'Valeria Moy')

https://www.milenio.com/opinion/valeria-moy/page/1
https://www.milenio.com/opinion/valeria-moy/page/2?
https://www.milenio.com/opinion/valeria-moy/page/3?
https://www.milenio.com/opinion/valeria-moy/page/4?
https://www.milenio.com/opinion/valeria-moy/page/5?
https://www.milenio.com/opinion/valeria-moy/page/6?
https://www.milenio.com/opinion/valeria-moy/page/7?
https://www.milenio.com/opinion/valeria-moy/page/8?
https://www.milenio.com/opinion/valeria-moy/page/9?
https://www.milenio.com/opinion/valeria-moy/page/10?
https://www.milenio.com/opinion/valeria-moy/page/11?
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div/div[3]/div/div[2]/div[2]/a[2]"}
  (Session info: headless chrome=89.0.4389.82)

Procesando: 1 de 99
Procesando: 2 de 99
Procesando: 3 de 99
Procesando: 4 de 99
Procesando: 5 de 99
Procesando: 6 de 99
Procesando: 7 de 99
Procesando: 8 de 99
Procesando: 9 de 99
Procesando: 10 de 99
Procesando: 11 de 99
Procesando: 12 de

In [15]:
df_val = pd.concat([elf_val, uni_val, mil_val], axis = 0)
df_val

,author,title,date,body,source,link
0,Valeria Moy,La polarización como estrategia,26/09/2018,Llevamos diez días hablando sobre un término u...,El Financiero,https://www.elfinanciero.com.mx/opinion/valeri...
1,Valeria Moy,Las palabras no se las lleva el viento,19/09/2018,"El presidente electo, Andrés Manuel López Obra...",El Financiero,https://www.elfinanciero.com.mx/opinion/valeri...
2,Valeria Moy,Entre trenes y aeropuertos,12/09/2018,"México, la economía número 11 en producción aj...",El Financiero,https://www.elfinanciero.com.mx/opinion/valeri...
3,Valeria Moy,Apuntes sobre el Informe,05/09/2018,El Sexto y último Informe de Gobierno del pres...,El Financiero,https://www.elfinanciero.com.mx/opinion/valeri...
4,Valeria Moy,Luces y sombras del acuerdo\n,29/08/2018,"\rEl lunes pasado se anunció, de una manera pa...",El Financiero,https://www.elfinanciero.com.mx/opinion/valeri...
...,...,...,...,...,...,...
94,Valeria Moy,Pero qué necesidad,20.11.2018/01:48,"\nLa idea no era parafrasear a Juan Gabriel, p...",Milenio,https://www.milenio.com/opinion/valeria-moy/pe...
95,Valeria Moy,La forma y el fondo,13.11.2018/02:19,\nEl jueves pasado el senador Ricardo Monreal ...,Milenio,https://www.milenio.com/opinion/valeria-moy/pe...
96,Valeria Moy,Abróchense los cinturones,06.11.2018/01:20,"\nNo, no es solo el aeropuerto. No es solo la ...",Milenio,https://www.milenio.com/opinion/valeria-moy/pe...
97,Valeria Moy,,,,Milenio,https://www.milenio.com/politica/el-mexico-que...


In [16]:
# Revisar texto con carácteres raros de El Financiero
df_val.to_csv('ValeriaMoy.csv')